<a href="https://colab.research.google.com/github/AnnetVasileva/machineLearning/blob/main/KDDCUp98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_set = '/content/drive/MyDrive/ML_ECO/cup98lrn.csv' # zip
valid_set = '/content/drive/MyDrive/ML_ECO/cup98val.csv'
val_target = '/content/drive/MyDrive/ML_ECO/valtarget.csv'

In [ ]:
df = pd.read_csv(train_set,low_memory = False)  #compression='zip'

In [ ]:
val = pd.read_csv(valid_set,low_memory = False)  #ASSUMING THE ORDER of columns is the same
valtarget = pd.read_csv(val_target,low_memory = False)

In [ ]:
valtarget

,CONTROLN,TARGET_B,TARGET_D
0,3,0,0.0
1,6,0,0.0
2,9,0,0.0
3,11,0,0.0
4,16,0,0.0
...,...,...,...
96362,191769,0,0.0
96363,191771,0,0.0
96364,191773,0,0.0
96365,191774,0,0.0


In [ ]:
val.CONTROLN

0        188946
1        126296
2        155244
3        123985
4        119118
          ...  
96362     51744
96363     78942
96364     82897
96365    110328
96366    123497
Name: CONTROLN, Length: 96367, dtype: int64

## Preprosessing
Our preprosessing will include 4 steps:
1. We will delete the columns we will not use (please see documentation to see which columns we drop)
2. Re-code /transform complex variables to easier ones
3. Encode ordinal variables using ordinal encoder and create one-hot-encoding where ordinal encoding is not suitable
4. Delete observations with missing data or additionally drop features that contain more than 50% of missing data

After each step we will check whether our target variable (TARGET_B) still has enough positive examples (value = 1)




**STEP 1: Drop Features**

In dropping features we follow certain rules, namely:
1. We drop every feature that is a date
2. We drop features that refer to the sources of information 
3. We drop features that are already covered implicitly by the other features that we will encode 
4. We drop features that have a very complex structure and would require a lot of dummy variables (like STATE)
5. We drop features that highly depend on features that we previously decided to drop (like WEALTH2)



In [ ]:
# some important stuff 
yesno_variables = df.columns[56:74]
rfas = df.columns[386:408] # RFA variables

In [ ]:
drop1 = df.columns[1:15].to_list() # need to drop these
drop2 = df.columns[16]#.to_list() # CLUSTER   
drop3 = df.columns[18]#.to_list() # AGEFLAG
drop4 = df.columns[44:51].to_list() # connection to the war and war veterans , might actually be relevant even though very specific data
drop11 = df.columns[51:54].to_list() # SOLICIT variables + MAJOR
drop5 = df.columns[54:56].to_list() 
drop6 = df.columns[362:386].to_list() # ADATES and RFA_XX
drop7 = df.columns[413:457].to_list() # Dates of different donations received
drop8 = df.columns[473]
drop9 = df.columns[480:482].to_list() # CLUSTER and GEOCODE
drop10 = df.columns[43] # drop the source from where the data is collected
drop12 = df.columns[74] #  LIFE STYLE DATA SOURCE
drop13 = df.columns[461] # Date associated with the smallest gift to date
drop14 = df.columns[463] # Date associated with the largest gift to date
drop15 = df.columns[465:468].to_list() # Dates of different gifts received
drop16 = df.columns[409] # MAXADATE
drop17 = df.columns[24] # Number of Children, very ambiguous since many empty cells, covered by other variables implicitly
drop18 = df.columns[196:199] # these are very weird codes, I do not know how to use them for the prediction
drop = drop1+drop5+drop6+drop7+drop9+drop11+drop15+drop4
df = df.drop(drop, axis = 1)
df = df.drop(drop2,axis = 1)
df = df.drop(drop3, axis = 1)
df = df.drop(drop8, axis = 1)
df = df.drop(drop10, axis = 1)
df = df.drop(drop12, axis = 1)
df = df.drop(drop13, axis = 1)
df = df.drop(drop14, axis = 1)
df = df.drop(drop17, axis = 1)


# repeat for val

val = val.drop(drop, axis = 1)
val = val.drop(drop2,axis = 1)
val = val.drop(drop3, axis = 1)
val = val.drop(drop8, axis = 1)
val = val.drop(drop10, axis = 1)
val = val.drop(drop12, axis = 1)
val = val.drop(drop13, axis = 1)
val = val.drop(drop14, axis = 1)
val = val.drop(drop17, axis = 1)


**STEP 2 : Encode**

Now we start encoding things
1. We first rename the first column to Index
2. We then extract the first letter from Domain to then encode it to one-hot encoding
3. Afterwards we change the CHILDXX variables , which will be also encoded as one-hot 
4.
5.

In [ ]:
df = df.rename(columns = {'Unnamed: 0' : 'Index'}) # rename first column to the Index
val = val.rename(columns = {'Unnamed: 0' : 'Index'}) # rename first column to the Index

In [ ]:
df.DOMAIN = df.DOMAIN.replace({' ': '99'}) # replace empty cells to 99 and then every 9 will be replaced to NaN

df['SES'] = 0 # NEW variable - socioeconomic status - second byte from DOMAIN
index = 0
for x in df.DOMAIN:
    df.SES[index] = int(df.DOMAIN[index][1])
    index += 1


# extracting the first 
index = 0
for x in df.DOMAIN:
    df.DOMAIN[index] = df.DOMAIN[index][0]
    index += 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [ ]:
val.DOMAIN = val.DOMAIN.replace({' ': '99'}) # replace empty cells to 99 and then every 9 will be replaced to NaN

val['SES'] = 0 # NEW variable - socioeconomic status - second byte from DOMAIN
index = 0
for x in val.DOMAIN:
    val.SES[index] = int(val.DOMAIN[index][1])
    index += 1


# extracting the first 
index = 0
for x in val.DOMAIN:
    val.DOMAIN[index] = val.DOMAIN[index][0]
    index += 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [ ]:
 df[rfas] = df[rfas].replace({' ': '999'})
 df['RFA_23'] = df['RFA_23'].replace({'1D': '91D','3E': '93E', '2D' : '92D', '1E': '91E', '1C': '91C', '1G': '91G', '2E': '92E',
                                     '1F': '91F','4E': '94E', '3F': '93F' , '2F' : '92F', '1B': '91B'})
  
for item in rfas:
    df[item + 'R'] = [ letter[0] for letter in df[item].to_list() ] # every first letter goes to Recency
    df[item + 'F'] = [ int(letter[1]) for letter in df[item].to_list() ] # every second letter (actually integer) goes to Frequency
    df[item + 'A'] = [ letter[2] for letter in df[item].to_list() ] # every third letter goes to Amount
  
for item in rfas:
    df[item + 'R'] = df[item + 'R'].replace({'9': np.NaN, 'U': np.NaN, 'P': np.NaN})
    df[item + 'F'] = df[item + 'F'].replace({9: np.NaN})
    df[item + 'A'] = df[item + 'A'].replace({'9': np.NaN, 'U': np.NaN, 'P': np.NaN})
 

In [ ]:
val[rfas] = val[rfas].replace({' ': '999'})
val['RFA_23'] = val['RFA_23'].replace({'1D': '91D','3E': '93E', '2D' : '92D', '1E': '91E', '1C': '91C', '1G': '91G', '2E': '92E',
                                     '1F': '91F','4E': '94E', '3F': '93F' , '2F' : '92F', '1B': '91B', '2C': '92C', '2G': '92G'})
  
for item in rfas:
    val[item + 'R'] = [ letter[0] for letter in val[item].to_list() ] # every first letter goes to Recency
    val[item + 'F'] = [ int(letter[1]) for letter in val[item].to_list() ] # every second letter (actually integer) goes to Frequency
    val[item + 'A'] = [ letter[2] for letter in val[item].to_list() ] # every third letter goes to Amount
  
for item in rfas:
    val[item + 'R'] = val[item + 'R'].replace({'9': np.NaN, 'U': np.NaN, 'P': np.NaN})
    val[item + 'F'] = val[item + 'F'].replace({9: np.NaN})
    val[item + 'A'] = val[item + 'A'].replace({'9': np.NaN, 'U': np.NaN, 'P': np.NaN})
 

In [ ]:
df.HOMEOWNR = df.HOMEOWNR.replace({'H': 1, 'U': 0, ' ': np.NaN})
df['CHILD03'] = df['CHILD03'].replace({' ': 1, 'B': 2, 'F': 3, 'M':4}) # we will assume that no indication of 
# children means no children as a category
df['CHILD07'] = df['CHILD07'].replace({' ': 1, 'B': 2, 'F': 3, 'M':4})	
df['CHILD12'] = df['CHILD12'].replace({' ': 1, 'B': 2, 'F': 3, 'M':4})
df['CHILD18'] = df['CHILD18'].replace({' ': 1, 'B': 2, 'F': 3, 'M':4})
#df.MAJOR = df.MAJOR.replace({'X': 1, ' ': 0})
df.PEPSTRFL = df.PEPSTRFL.replace({'X': 1, ' ':0})
df.DOMAIN = df.DOMAIN.replace({9 : np.NaN, '9': np.NaN})
df.SES = df.SES.replace({9 : np.NaN, 4: 3}) # please read documentation, the decision was made to replace every 4 to 3 so that technically 3 will include 
# all lowest SES
df.SES = df.SES.replace({3: 1, 1: 3}) # now we replace all 3 with 1 and all 1 with 3 so that we can decode it ordinally
df.GENDER = df.GENDER.replace({'A': 'U', 'C': 'U', ' ': np.NaN, 'J' : 'U'})
#df.MDMAUD_A = df.MDMAUD_A.replace({'X': np.NaN})
df.MDMAUD_F = df.MDMAUD_F.replace({'X': np.NaN})
df.DOMAIN = df.DOMAIN.replace({'S': 1, 'T':2, 'R':3, 'C':4, 'U':5})

for item in yesno_variables:
    df[item] = df[item].replace({' ': 0, 'N':0, 'Y': 1}) # these data values are a bit ambiguous since some of them do not have real N so we assume
    #that empty cells represent negative observations


recency = df.columns[376:442].to_list()[0::3]+ ['RFA_2R']
frequency = df.columns[376:442].to_list()[1::3]+ ['RFA_2F']
amount = df.columns[376:442].to_list()[2::3] + ['RFA_2A']


In [ ]:
val.HOMEOWNR = val.HOMEOWNR.replace({'H': 1, 'U': 0, ' ': np.NaN})
val['CHILD03'] = val['CHILD03'].replace({' ': 'N'}) # we will assume that no indication of 
# children means no children as a category
val['CHILD07'] = val['CHILD07'].replace({' ': 'N'})	
val['CHILD12'] = val['CHILD12'].replace({' ': 'N'})
val['CHILD18'] = val['CHILD18'].replace({' ': 'N'})
#df.MAJOR = df.MAJOR.replace({'X': 1, ' ': 0})
val.PEPSTRFL = val.PEPSTRFL.replace({'X': 1, ' ':0})
val.DOMAIN = val.DOMAIN.replace({9 : np.NaN, '9': np.NaN})
val.SES = val.SES.replace({9 : np.NaN, 4: 3}) # please read documentation, the decision was made to replace every 4 to 3 so that technically 3 will include 
# all lowest SES
val.SES = val.SES.replace({3: 1, 1: 3}) # now we replace all 3 with 1 and all 1 with 3 so that we can decode it ordinally
val.GENDER = val.GENDER.replace({'A': 'U', 'C': 'U', ' ': np.NaN, 'J' : 'U'})
#df.MDMAUD_A = df.MDMAUD_A.replace({'X': np.NaN})
val.MDMAUD_F = val.MDMAUD_F.replace({'X': np.NaN})
val.DOMAIN = val.DOMAIN.replace({'S': 1, 'T':2, 'R':3, 'C':4, 'U':5})

for item in yesno_variables:
    val[item] = val[item].replace({' ': 0, 'N':0, 'Y': 1}) # these data values are a bit ambiguous since some of them do not have real N so we assume
    #that empty cells represent negative observations


recency_val = val.columns[374:442].to_list()[0::3] + ['RFA_2R']
frequency_val = val.columns[374:442].to_list()[1::3] + ['RFA_2F']
amount_val = val.columns[374:442].to_list()[2::3] + ['RFA_2A']

In [ ]:
too_many_nas = [] # columns where we still have too many nans

for item in df:
    if df[item].isna().sum() > len(df)*0.4:
      too_many_nas.append(str(item))

df = df.drop(too_many_nas, axis = 1)
df = df.drop('Index', axis = 1)

In [ ]:
too_many_nas_val = [] # columns where we still have too many nans

for item in val:
    if val[item].isna().sum() > len(df)*0.4:
      too_many_nas_val.append(str(item))

if too_many_nas_val != too_many_nas:
  too_many_nas_val = too_many_nas
else:
    pass


val = val.drop(too_many_nas_val, axis = 1)
val = val.drop('Index', axis = 1)

In [ ]:
print(val.shape)
print(df.shape)

(96367, 410)
(95412, 412)


In [ ]:
# now we need to drop 
df = df.drop(rfas, axis = 1)

recency_dropped = list(set.intersection(set(too_many_nas), set(recency)))
frequency_dropped = list(set.intersection(set(too_many_nas), set(frequency)))
amount_dropped = list(set.intersection(set(too_many_nas), set(amount)))

recency = list(np.setdiff1d(recency,recency_dropped))
frequency = list(np.setdiff1d(frequency,frequency_dropped))
amount = list(np.setdiff1d(amount,amount_dropped))

freq = [[1.0,2.0,3.0,4.0]] *len(frequency)
am = [['A', 'B', 'C', 'D', 'E', 'F', 'G']] * len(amount)

In [ ]:
print(val.shape)
print(df.shape)

(96367, 410)
(95412, 390)


In [ ]:
# now we need to drop 
val = val.drop(rfas, axis = 1)

recency_dropped_val = list(set.intersection(set(too_many_nas_val), set(recency_val)))
frequency_dropped_val = list(set.intersection(set(too_many_nas_val), set(frequency_val)))
amount_dropped_val = list(set.intersection(set(too_many_nas_val), set(amount_val)))

recency_val = list(np.setdiff1d(recency_val,recency_dropped_val))
frequency_val = list(np.setdiff1d(frequency_val,frequency_dropped_val))
amount_val = list(np.setdiff1d(amount_val,amount_dropped_val))

freq = [[1.0,2.0,3.0,4.0]] *len(frequency)
am = [['A', 'B', 'C', 'D', 'E', 'F', 'G']] * len(amount)

In [ ]:
print(val.shape)
print(df.shape)

(96367, 388)
(95412, 390)


**Step 3: One Hot Encoding**

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
#dropna
df = df.dropna()
val = val.dropna()

one_hot_encoding = ['DOMAIN', 'CHILD03','CHILD12','CHILD18', 'GENDER', 'MDMAUD_R'] + recency
categorical_encoding = ['MDMAUD_A', 'SES', 'INCOME'] + frequency + amount
categories = [[ 'X' ,'L', 'C', 'M', 'T'],
              ['1','2','3'],
              [1.,2.,3.,4.,5.,6.,7.]] + freq + am


ordinal_encoder = OrdinalEncoder(categories = categories)
one_hot_encoder = OneHotEncoder()

df_onehot = one_hot_encoder.fit_transform(df[one_hot_encoding])
df_ordinal = ordinal_encoder.fit_transform(df[categorical_encoding])

val_onehot = one_hot_encoder.fit_transform(val[one_hot_encoding])
val_ordinal = ordinal_encoder.fit_transform(val[categorical_encoding])


In [ ]:
len(one_hot_encoding)


24

In [ ]:
print(df[one_hot_encoding[0]].unique())
print(df[one_hot_encoding[1]].unique())
print(df[one_hot_encoding[2]].unique())
print(df[one_hot_encoding[3]].unique())
print(df[one_hot_encoding[4]].unique())
print(df[one_hot_encoding[5]].unique())
print(df[one_hot_encoding[6]].unique())
print(df[one_hot_encoding[7]].unique())
print(df[one_hot_encoding[8]].unique())
print(df[one_hot_encoding[9]].unique())
print(df[one_hot_encoding[10]].unique())
print(df[one_hot_encoding[11]].unique())
print(df[one_hot_encoding[12]].unique())
print(df[one_hot_encoding[13]].unique())
print(df[one_hot_encoding[14]].unique())
print(df[one_hot_encoding[15]].unique())
print(df[one_hot_encoding[16]].unique())
print(df[one_hot_encoding[17]].unique())
print(df[one_hot_encoding[18]].unique())
print(df[one_hot_encoding[19]].unique())
print(df[one_hot_encoding[20]].unique())
print(df[one_hot_encoding[21]].unique())
print(df[one_hot_encoding[22]].unique())
print(df[one_hot_encoding[23]].unique())


[1. 2. 3. 4. 5.]
[1 4 3 2]
[1 3 4 2]
[1 3 4 2]
['F' 'M' 'U']
['X' 'D' 'L' 'C' 'I']
['L' 'S' 'A' 'I' 'N' 'F']
['A' 'S' 'L' 'I' 'N' 'F']
['A' 'S' 'L' 'I' 'N' 'F']
['L' 'S' 'A' 'N' 'F' 'I']
['L' 'S' 'A' 'N' 'I' 'F']
['A' 'S' 'L' 'N' 'I' 'F']
['A' 'S' 'L' 'N' 'I' 'F']
['A' 'S' 'L' 'N' 'F' 'I']
['I' 'S' 'L' 'A' 'N' 'F']
['A' 'S' 'L' 'N' 'F']
['L']
['A' 'S' 'L' 'F' 'N']
['A' 'S' 'N' 'L' 'F']
['A' 'S' 'L' 'N']
['A' 'S' 'L' 'F' 'I' 'N']
['I' 'S' 'A' 'L' 'N' 'F']
['A' 'S' 'L' 'I' 'N']
['A' 'S' 'L' 'I' 'N' 'F']


In [ ]:
print(val[one_hot_encoding[0]].unique())
print(val[one_hot_encoding[1]].unique())
print(val[one_hot_encoding[2]].unique())
print(val[one_hot_encoding[3]].unique())
print(val[one_hot_encoding[4]].unique())
print(val[one_hot_encoding[5]].unique())
print(val[one_hot_encoding[6]].unique())
print(val[one_hot_encoding[7]].unique())
print(val[one_hot_encoding[8]].unique())
print(val[one_hot_encoding[9]].unique())
print(val[one_hot_encoding[10]].unique())
print(val[one_hot_encoding[11]].unique())
print(val[one_hot_encoding[12]].unique())
print(val[one_hot_encoding[13]].unique())
print(val[one_hot_encoding[14]].unique())
print(val[one_hot_encoding[15]].unique())
print(val[one_hot_encoding[16]].unique())
print(val[one_hot_encoding[17]].unique())
print(val[one_hot_encoding[18]].unique())
print(val[one_hot_encoding[19]].unique())
print(val[one_hot_encoding[20]].unique())
print(val[one_hot_encoding[21]].unique())
print(val[one_hot_encoding[22]].unique())
print(val[one_hot_encoding[23]].unique())

[4. 2. 1. 5. 3.]
['N' 'M' 'F' 'B']
['N' 'F' 'M' 'B']
['N' 'M' 'F' 'B']
['M' 'F' 'U']
['X' 'D' 'L' 'I' 'C']
['A' 'S' 'I' 'L' 'N' 'F']
['A' 'S' 'L' 'I' 'N']
['A' 'S' 'L' 'I' 'N' 'F']
['A' 'S' 'L' 'F' 'N']
['A' 'S' 'L' 'N' 'I' 'F']
['A' 'S' 'L' 'N' 'I' 'F']
['A' 'S' 'L' 'N' 'I' 'F']
['A' 'S' 'N' 'L' 'I' 'F']
['A' 'S' 'N' 'L' 'F' 'I']
['A' 'S' 'N' 'L' 'F']
['L']
['A' 'S' 'L' 'N' 'F']
['A' 'S' 'F' 'N' 'L']
['A' 'S' 'L' 'N']
['A' 'S' 'L' 'N' 'I' 'F']
['A' 'S' 'L' 'I' 'N' 'F']
['A' 'S' 'L' 'I' 'N' 'F']
['A' 'S' 'L' 'I' 'N' 'F']


In [ ]:
print(val.shape)
print(df.shape)

(15776, 388)
(15578, 390)


In [ ]:
# TARGET VARIABLES
label_binary = df['TARGET_B'].to_numpy() # 1/0 donated or not
label_dollars = df['TARGET_D'].to_numpy() # how much donated


columns = np.setdiff1d(df.columns,one_hot_encoding)
columns = np.setdiff1d(columns,categorical_encoding)
columns = np.setdiff1d(columns,['TARGET_B','TARGET_D','CONTROLN'])

df = df[columns]

controln = val['CONTROLN']
columns = np.setdiff1d(val.columns,one_hot_encoding)
columns = np.setdiff1d(columns,categorical_encoding)
columns = np.setdiff1d(columns,['CONTROLN'])


val = val[columns]

In [ ]:
print(val.shape)
print(df.shape)

(15776, 322)
(15578, 322)


In [ ]:
df

,AC1,AC2,ADI,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,AGE,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,ANC1,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,AVGGIFT,...,POP90C4,POP90C5,RAMNTALL,RHP1,RHP2,RHP3,RHP4,RP1,RP2,RP3,RP4,SEC1,SEC2,SEC3,SEC4,SEC5,STEREO,TIMELAG,TPE1,TPE10,TPE11,TPE12,TPE13,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,VC1,VC2,VC3,VC4,VETERANS,VOC1,VOC2,VOC3,WALKER
4,7,11,127.0,1,1,1,15,28,4,78.0,33,45,50,36,46,50,27,14,21,13,15,20,12,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.864865,...,46,54,254.0,52,53,16,6,25,58,74,83,2,26,1,20,7,1,14.0,73,24,27,3,76,22,1,1,0,0,0,3,1,9,16,53,20,0,90,45,18,1
28,2,3,201.0,0,0,0,13,28,0,49.0,30,38,39,29,39,41,36,9,25,39,17,5,4,1,0,1,1,0,1,0,0,12,1,8,0,0,7,1,0,8.500000,...,51,49,102.0,64,67,19,4,94,99,99,99,5,28,5,26,2,0,9.0,80,22,33,17,57,12,2,2,0,0,0,0,6,50,8,19,9,0,99,68,10,0
33,5,6,127.0,0,0,0,7,15,1,69.0,33,36,41,40,43,47,11,14,31,16,9,10,9,10,0,3,0,1,1,0,0,4,0,6,0,0,3,3,0,16.923077,...,46,54,220.0,34,36,10,5,43,96,98,98,10,13,0,3,20,0,3.0,70,23,27,4,75,12,11,1,9,0,0,5,2,16,13,52,23,0,91,31,4,0
38,5,3,107.0,0,0,0,16,33,1,75.0,28,35,41,32,39,44,22,21,27,19,16,8,6,3,0,1,0,0,0,5,0,1,0,17,0,0,1,2,4,12.000000,...,48,52,96.0,54,50,13,4,11,89,90,92,3,23,3,11,12,0,12.0,86,20,24,3,68,8,4,4,0,0,0,0,2,19,16,19,23,0,94,55,17,0
43,6,4,127.0,0,0,0,15,28,1,44.0,33,39,41,31,41,43,30,10,22,33,20,9,3,2,0,3,0,1,1,0,0,3,0,8,0,1,6,8,0,17.000000,...,50,50,68.0,67,68,18,4,70,73,77,79,5,26,4,21,6,0,12.0,87,32,34,3,89,9,0,0,0,0,0,2,2,36,20,19,12,0,99,86,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95399,8,5,199.0,0,0,0,11,23,0,43.0,34,42,45,36,45,48,25,13,20,23,16,13,10,6,0,0,0,0,0,0,0,11,0,5,0,0,13,1,0,16.235294,...,49,51,276.0,53,55,14,4,5,7,26,79,2,24,2,19,5,1,21.0,71,19,22,0,77,21,0,0,0,0,0,2,6,30,8,20,13,0,89,67,19,0
95400,5,6,127.0,1,2,0,17,37,1,86.0,38,46,49,40,50,53,24,9,18,21,15,11,10,18,1,1,0,1,1,0,0,8,0,8,0,0,7,2,0,12.125000,...,45,55,97.0,62,54,14,5,49,50,53,62,5,20,1,17,7,0,28.0,82,28,30,7,73,10,2,0,1,0,0,3,3,36,19,32,15,0,82,55,20,0
95404,5,6,211.0,0,0,0,11,23,1,53.0,31,39,43,34,43,47,28,16,22,22,14,11,9,6,0,0,0,0,1,0,0,5,1,3,0,0,7,1,0,5.900000,...,49,51,59.0,49,52,16,5,1,3,8,32,1,26,1,21,4,1,3.0,78,18,23,5,59,15,1,0,0,0,0,4,3,32,22,29,10,0,83,47,14,0
95410,6,3,13.0,0,0,0,13,28,1,58.0,34,42,45,36,45,49,25,12,21,21,18,12,7,9,0,2,0,0,0,0,0,3,1,9,0,0,2,6,1,12.146341,...,47,53,498.0,65,68,18,5,88,88,90,91,6,29,4,21,10,0,4.0,80,18,28,11,52,11,0,0,0,0,1,2,6,35,18,20,8,0,99,85,43,0


In [ ]:
df[df.columns[300:322]]#.to_numpy()[:, np.any(df == 'N', axis = 0)]

,TPE1,TPE10,TPE11,TPE12,TPE13,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,VC1,VC2,VC3,VC4,VETERANS,VOC1,VOC2,VOC3,WALKER
4,73,24,27,3,76,22,1,1,0,0,0,3,1,9,16,53,20,0,90,45,18,1
28,80,22,33,17,57,12,2,2,0,0,0,0,6,50,8,19,9,0,99,68,10,0
33,70,23,27,4,75,12,11,1,9,0,0,5,2,16,13,52,23,0,91,31,4,0
38,86,20,24,3,68,8,4,4,0,0,0,0,2,19,16,19,23,0,94,55,17,0
43,87,32,34,3,89,9,0,0,0,0,0,2,2,36,20,19,12,0,99,86,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95399,71,19,22,0,77,21,0,0,0,0,0,2,6,30,8,20,13,0,89,67,19,0
95400,82,28,30,7,73,10,2,0,1,0,0,3,3,36,19,32,15,0,82,55,20,0
95404,78,18,23,5,59,15,1,0,0,0,0,4,3,32,22,29,10,0,83,47,14,0
95410,80,18,28,11,52,11,0,0,0,0,1,2,6,35,18,20,8,0,99,85,43,0


In [ ]:
df = np.concatenate((df,df_onehot.toarray(), df_ordinal), axis = 1)
val = np.concatenate((val,val_onehot.toarray(), val_ordinal), axis = 1)

In [ ]:
print(val.shape)
print(df.shape)

(15776, 484)
(15578, 485)


In [ ]:
df[:, 285]
#label_binary.shape

array([52., 64., 34., ..., 49., 65., 65.])

In [ ]:
val.shape

(15776, 484)

In [ ]:
# FINAL RESULT SHOULD BE df and two labels
import pickle
data = {}
data['X'] = val
#data['y'] = label_binary
data['CONTROLN'] = controln
pickle_path = '/content/drive/MyDrive/ML_ECO/val_set.pkl'
with open(pickle_path, "wb") as f:
    pickle.dump(data, f)